# Telco Classification Project

***

PLAN -> Acquire -> Prepare -> Explore -> Model & Evaluate -> Deliver

# Project Planning

 Customers at the telecommunications company, Telco, are churning.  The goal of this project is to determine why customers are churning by developing machine learning classification models to predict customer churn based on the customer data available. The models will be created using Python, Pandas, Matplot, Seaborn, and Scikit-Learn libraries.

### Business Goals

- Find drivers for customer churn at Telco. 


- Construct a Machine Learning classification model that accurately predicts customer churn.

***

# Questions to Consider:

So customers are churning and we don't know why.  

- Are there clear groupings where a customer is more likely to churn?

- What if you consider contract type?
- Is there a tenure value at which month-to-month customers are most likely to churn? 1-year contract customers? 2-year contract customers?

Thoughts on what could be going on? (Be sure to state these thoughts not as facts but as untested hypotheses until you test them!). 

You might want to plot the rate of churn on a line chart where x is the tenure and y is the rate of churn (customers churned/total customers).
Are there features that indicate a higher likelihood for customer churn?

- How influential are internet service type, phone service type, online security and backup services, senior citizen status, paying more than x% of customers with the same services, etc.?
- Is there a price threshold for specific services beyond which the likelihood of churn increases?

- If so, what is that point and for which service(s)?
- If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

Clearly state your starting hypotheses (and add the testing of these to your task list).

### From the previous Telco churn project, I discovered that many month-to-month customers who churn are not on automatic payment plans.

To follow up:

- Who are the customers on automatic payments? Senior citizens, singles, couples, persons with dependents?
- What services do they subscribe to?
- How much do they pay compared to those not on automatic payments? Are they charged more?
- What is the tenure at which month-to-month customers not on automatic payments churn?  How does this compare to 1-year and 2-year contract non-automatic payment customers?
- Are there more 1-year or 2-year contract non-automatic payment type customers than month-to-month customers?





### Initial Hypothesis: 

- Customers not on automatic payment plans pay more than customers on automatic payment plans

-OR-

- Customers not on automatic payment plans have less services than customers on automatic payment plans




#### Data Acquisition

Plan -> ACQUIRE -> Prepare -> Explore -> Model & Evaluate -> Deliver

In [1]:
#import libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# https://seaborn.pydata.org/tutorial/color_palettes.html
# https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html

# set seaborn color palette
sns.color_palette("crest")
pal = sns.set_palette('crest')

# import models
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import classification_report, confusion_matrix, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import sklearn.feature_selection as feat_select
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder, normalize

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

#import custom functions
import acquire
import prepare

In [2]:
# Acquire data from SQL Telco Churn database using custom function

df = acquire.get_telco_data()

In [ ]:
# Take an initial look at the data acquired
df.info() 

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
#Check for nulls
df.isna().sum()

In [ ]:
# Check out distributions of numeric columns.

num_cols = df.columns[[df[col].dtype == 'int64' for col in df.columns]]
for col in num_cols:
    plt.hist(df[col])
    plt.title(col)
    plt.show()

In [ ]:
# Use .describe with object columns.

obj_cols = df.columns[[df[col].dtype == 'O' for col in df.columns]]
for col in obj_cols:
    print(df[col].value_counts())
    print(df[col].value_counts(normalize=True, dropna=False))
    print('----------------------')

In [ ]:
# Find columns with missing values and the total of missing values.

missing = df.isnull().sum()
missing[missing > 0]

In [ ]:
# total churn counts
df.churn.value_counts()

# Data Preparation

Plan -> Acquire -> PREPARE -> Explore -> Model & Evaluate -> Deliver
***

### In Your prepare.py module

Store functions that are needed to prepare your data; make sure your module contains the necessary imports to run your code. Your final function should do the following:

Split your data into train/validate/test.

Handle Missing Values.

Handle erroneous data and/or outliers you wish to address.

Encode variables as needed.

Create any new features, if you decided to make any for this project.

### In Your Notebook

Explore missing values and document takeaways/action plans for handling them.

Is 'missing' equivalent to 0 (or some other constant value) in the specific case of this variable?

Should you replace the missing values with a value it is most likely to represent, like mean/median/mode?

Should you remove the variable (column) altogether because of the percentage of missing data?

Should you remove individual observations (rows) with a missing value for that variable?

Explore data types and adapt types or data values as needed to have numeric represenations of each attribute.

Create any new features you want to use in your model. Some ideas you might want to explore after securing a MVP:

Create a new feature that represents tenure in years.

Create single variables for or find other methods to merge variables representing the information from the following columns:

phone_service and multiple_lines
dependents and partner
streaming_tv & streaming_movies
online_security & online_backup
Import your prepare function from your prepare.py module and use it to prepare your data in your notebook.



In [3]:
# Prepare Telco data by removing ID number columns, removing any duplicates, encoding and creating dummies
df = prepare.prep_telco(df)

['multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'contract_type', 'internet_service_type', 'payment_type']
Index(['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'internet_service_type_id', 'contract_type_id',
       'paperless_billing', 'payment_type_id', 'monthly_charges',
       'total_charges', 'churn', 'multiple_lines_No',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'online_security_No', 'online_security_No internet service',
       'online_security_Yes', 'online_backup_No',
       'online_backup_No internet service', 'online_backup_Yes',
       'device_protection_No', 'device_protection_No internet service',
       'device_protection_Yes', 'tech_support_No',
       'tech_support_No internet service', 'tech_support_Yes',
       'streaming_tv_No', 'streaming_tv_No internet service',
       'streaming_tv_Yes', 'streaming_movies_No',
       

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28084 entries, 0 to 28171
Data columns (total 57 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   gender                                  28084 non-null  int64  
 1   senior_citizen                          28084 non-null  int64  
 2   partner                                 28084 non-null  int64  
 3   dependents                              28084 non-null  int64  
 4   tenure                                  28084 non-null  int64  
 5   phone_service                           28084 non-null  int64  
 6   internet_service_type_id                28084 non-null  int64  
 7   contract_type_id                        28084 non-null  int64  
 8   paperless_billing                       28084 non-null  int64  
 9   payment_type_id                         28084 non-null  int64  
 10  monthly_charges                         28084 non-null  fl

In [5]:
df.shape

(28084, 57)

In [11]:
df.describe()

,gender,senior_citizen,partner,dependents,tenure,phone_service,internet_service_type_id,contract_type_id,paperless_billing,payment_type_id,...,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
count,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,...,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.00000,28084.00000,28084.00000,28084.00000
mean,0.504344,0.162512,0.484546,0.300527,32.469449,0.902863,1.870816,1.692636,0.592651,2.317476,...,0.548782,0.209799,0.241419,0.344538,0.440108,0.215354,0.25000,0.25000,0.25000,0.25000
std,0.499990,0.368927,0.499770,0.458495,24.533654,0.296150,0.737036,0.834118,0.491350,1.149901,...,0.497623,0.407172,0.427951,0.475226,0.496409,0.411075,0.43302,0.43302,0.43302,0.43302
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,1.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
50%,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,2.000000,1.000000,1.000000,2.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,2.000000,2.000000,1.000000,3.000000,...,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.25000,0.25000,0.25000,0.25000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,3.000000,3.000000,1.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.00000,1.00000


In [12]:
#Split the data to ready for exploration, modeling, and testing
# Using the prepare custom library, take in a DataFrame and return train, validate, and test DataFrames; 
# stratify on churn

train, validate, test = prepare.split_data(df)

In [13]:
train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,internet_service_type_id,contract_type_id,paperless_billing,payment_type_id,...,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
24440,1,0,0,0,4,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,0
20315,0,0,1,1,40,1,1,1,1,2,...,1,0,0,1,0,0,0,0,1,0
22126,1,0,0,0,6,1,3,1,0,3,...,1,0,0,0,0,1,0,0,0,1
22803,1,0,1,0,50,1,2,2,1,1,...,0,1,0,0,1,0,0,0,1,0
7861,1,1,1,0,41,1,2,1,1,1,...,1,0,0,0,1,0,1,0,0,0


### Takeaways

- Removed duplicate index column created when acquired
- Removed customer_id since it is not helpful
- Removed duplicate ID columns created when tables were joined

- encoded all columns containing strings for: 'partner','dependents','phone_service','multiple_lines','paperless_billing','churn','gender'
- create dummy variables for: 'multiple_lines','online_security','online_backup','device_protection','tech_support','streaming_tv','streaming_movies','contract_type','internet_service_type','payment_type'